# Complexity analysis - Practical session 2: NP Completness and the backpack/knapsack problem

In this TP we propose to examine different ways of coding and solving problems of the NP class. We are interested in the problem of the backpack.
We suppose that we have $l$ items each having a utility (or gain) $u_i$. Each of these items has a weight $m_i$. We try to maximize the gain by packing
as many items as possible in a maximum capacity bag $M$. We distinguish two interesting cases :
 - the items are only available in one copy, i.e. we are trying to determine the quantity $x_i \in \{0,1\}$ associated with each item, 
 - one can take several times the same item, i.e. $x_i \in \mathbb{N}^+$. The problem is formalized as follows:
\begin{aligned}
U & =  \text{max}_{x_i} \sum_i x_i u_i \\\
    & \text{s.c.} \sum_i x_i m_i \leq M
\end{aligned}

We will examine different methods to solve this problem allowing you to feel its complexity. You generate for each test that you will make a utility vector and a vector 
of weights that will be randomly drawn integers in $[1.10]$. You will set $M$ according to the number of possible items, for example if you have $n$ items (which will be a 
parameter of your test procedure), you will be able to choose $M=7n$. You will write a function `solve_bag` for each variant that will take the utility, weight and $M$ vectors as parameters and 
will return the maximum value (total gain) reached, as well as the time related to the calculation. 

Write the code below that will generate the possible data for this problem.


In [9]:
import random

n = 5
M = 7 * n
utility = [random.randint(1, 10) for i in range(0, n)]
weight = [random.randint(1, 10) for i in range(0, n)]

print("Utility : ", utility)
print("Weight : ", weight)

Utility :  [7, 6, 3, 9, 9]
Weight :  [8, 8, 10, 2, 1]


## Brute force approach

We don't bother with complex considerations here: write a method that calculates all possible combinations ($2^n$!), evaluates them, and returns the optimal gain. 


In [10]:
def brutForce(nb, weight, utility, index):
    if((index == 0) or (nb == 0)):
        return(0)
    if(weight[index - 1] > nb):
        return(brutForce(nb, weight, utility, index - 1))
    else:
        return(utility[index - 1] + brutForce(nb - weight[index - 1], weight, utility, index-1))

n = 5
M = 7 * n
utility = [random.randint(1, 10) for i in range(0, n)]
weight = [random.randint(1, 10) for i in range(0, n)]

print("Utility : ", utility)
print("Weight : ", weight)
print("Result : ", brutForce(M, weight, utility, n))

Utility :  [2, 3, 10, 8, 4]
Weight :  [1, 4, 9, 2, 1]
Result :  27


Same thing but this time you can choose the same item several times. For each item we can determine the maximum limit of the number of times we can choose this item as the integer part of 
$M/m_i$. Be careful, calculation times can become very long for high $M$ values.

In [11]:
def brutForce2(nb, weight, utility, index, maxUse):
    if((index <= 0) or (maxUse > index)):
        return(0)
    result = 0
    for i in range(index):
        if(nb - weight[i] >= 0):
            result = max(result, utility[i] + brutForce2(nb - weight[i], weight, utility, index, maxUse + 1))
    return(result)

n = 5
M = 7 * n
utility = [random.randint(1, 10) for i in range(0, n)]
weight = [random.randint(1, 10) for i in range(0, n)]

print("Utility : ", utility)
print("Weight : ", weight)
print("Result : ", brutForce2(M, weight, utility, n, 3))

Utility :  [9, 9, 4, 5, 1]
Weight :  [7, 2, 2, 10, 10]
Result :  27


## Greedy approach

For each object the gain/mass ratio ($u_i/m_i$) is calculated. The objects are sorted in descending order, then the bag is filled in this order until no more items can be added.
Compare the quality of the solution obtained with that of the previous exact solver. In particular, find cases where the gluttonous strategy does not give the optimal solution of the 
problem. Here again you will code two versions of the function (only one item available and unlimited number of items available). 


In [12]:
import numpy as np

def sortList(list1, list2):
    zippedPairs = zip(list2, list1)
    result = [x for _, x in sorted(zippedPairs, reverse = True)]
    return(result)

def greedyOneItem(nb, weight, utility, index):
    vect = np.array(utility) / np.array(weight)
    sortedWeight = sortList(weight, vect)
    sortedUtility = sortList(utility, vect)
    i = 0
    result = 0
    while((nb > 0) and (i < index)):
        if((nb - sortedWeight[i]) >= 0):
            result += sortedUtility[i]
            nb -= sortedWeight[i]
        i += 1
    return(result)

def greedyUnlimitedItems(nb, weight, utility, index, maxUse):
    vect = np.array(utility) / np.array(weight)
    sortedWeight = sortList(weight, vect)
    sortedUtility = sortList(utility, vect)
    result = 0
    for i in range(index):
        if((nb - weight[i]) >= 0):
            result = max(result, sortedUtility[i] + greedyUnlimitedItems(nb - sortedWeight[i], weight, utility, index, maxUse + 1))
    return(result)

print("Utility : ", utility)
print("Weight : ", weight)
print("Result greedyOneItem() : ", greedyOneItem(M, weight, utility, n))
print("Result greedyUnlimitedItems() : ", greedyUnlimitedItems(M, weight, utility, n, 0))

Utility :  [9, 9, 4, 5, 1]
Weight :  [7, 2, 2, 10, 10]
Result greedyOneItem() :  28
Result greedyUnlimitedItems() :  148


## Dynamic programming

We limit ourselves here to the case where only one item of each object is available.

The idea of dynamic programming is to solve incrementally simpler versions of the problem, and to store intermediate results necessary to 
add new variables. A time-space compromise is then made. In the case of the backpack problem, the problem is said to be with *optimal substructure*, that is to say that we 
can find the optimal value of the problem at variable $i$ from the optimal value at variable $i-1$. 
The following quantity $P(k,m)$, describing the state of the system for k variables, is defined by recurrence :
$$
\begin{eqnarray}
P(k,m) & = & \text{max}_{x_i} \sum^k_i x_i u_i \\
    &    & \text{s.c.} \sum_i x_i m_i \leq m
\end{eqnarray}
$$
then the optimal solution is either :
 - the optimal solution $P(k-1,m)$ where one chooses not to add the item, i.e. $x_k=0$.
- the optimal solution $P(k-1,m-m_k)+u_k$ where we choose to add the item, i.e. $x_k=1$.

You just have to build a table of the different possibilities $P(k,m)$. Once this table is built, you just have to start from the element 
$P(k,M)$ and go back to the $P(0,.)$ element to find out whether you choose the item or not and thus construct the solution.

We note then that the complexity of the algorithm is in time and space $o(nM)$. Although polynomial, we have not shown that $P=NP$: 
since the coding of $M$ is done on $log(M)$ bits, we remain within the exponential complexity of the size of the input.
  

Code and test this algorithm.

Optionnaly, you can get some help from the Wikipedia page on [0-1 knapasack problem](https://en.wikipedia.org/wiki/Knapsack_problem).

In [13]:
def dynamic(nb, weight, utility, index):
    result = [[0 for k in range(nb + 1)] for k in range(index + 1)]
    for i in range(index + 1):
        for j in range(nb + 1):
            if((i == 0) or (j == 0)):
                result[i][j] = 0
            elif(weight[i - 1] <= j):
                result[i][j] = max(result[i - 1][j - 1], utility[i - 1] + result[i - 1][j - weight[i - 1]])
            else:
                result[i][j] = result[i - 1][j]
    return(result[index][nb])

print("Utility : ", utility)
print("Weight : ", weight)
print("Result dynamic() : ", dynamic(M, weight, utility, n))

Utility :  [9, 9, 4, 5, 1]
Weight :  [7, 2, 2, 10, 10]
Result dynamic() :  28


## Summary and final conclusions
For each method, vary the number of items $n$, and measure an average time taken on $10$ resolution of the problem. Draw the corresponding average execution time curves
for the 3 methods in both cases (one or more times the same item, where dynamic programming will be excluded). 

In [15]:
import matplotlib as plt
import matplotlib.pyplot as plt
import time

plt.figure()
brutForceChart = []
brutForce2Chart = []
greedyOneItemChart = []
greedyUnlimitedItemsChart = []
dynamicChart = []
for i in range(1, n):
    averageBruteForce = 0
    averageBruteForce2 = 0
    averageGreedyOneItem = 0
    averageGreedyUnlimitedItems = 0
    averageDynamic = 0
    for j in range(20):
        t1 =  time.process_time()
        brutForce(M, weight, utility, n)
        t2 = time.process_time()
        averageBruteForce += t2 - t1
        t1 =  time.process_time()
        brutForce2(M, weight, utility, n, 0)
        t2 = time.process_time()
        averageBruteForce2 += t2 - t1
        t1 =  time.process_time()
        greedyOneItem(M, weight, utility, n)
        t2 = time.process_time()
        averageGreedyOneItem += t2 - t1
        t1 =  time.process_time()
        greedyUnlimitedItems(M, weight, utility, n, 0)
        t2 = time.process_time()
        averageGreedyUnlimitedItems += t2 - t1
        t1 =  time.process_time()
        dynamic(M, weight, utility, n)
        t2 = time.process_time()
        averageDynamic += t2 - t1
    brutForceChart.append(averageBruteForce / 20)
    brutForce2Chart.append(averageBruteForce2 / 20)
    greedyOneItemChart.append(averageGreedyOneItem / 20)
    greedyUnlimitedItemsChart.append(averageGreedyUnlimitedItems / 20)
    dynamicChart.append(averageDynamic / 20)
plt.plot(np.arange(1, n), brutForceChart, label = "brutForce")
plt.plot(np.arange(1, n), brutForce2Chart, label = "brutForce2")
plt.plot(np.arange(1, n), greedyOneItemChart, label = "greedyOneItem")
plt.plot(np.arange(1, n), greedyUnlimitedItemsChart, label = "greedyUnlimitedItems")
plt.plot(np.arange(1, n), dynamicChart, label = "dynamic")
plt.legend(box = (1.05, 1), loc = 'upper left', borders = 0.)
plt.show()